## Creando Macros anidadas de Roll20

Primero, he aquí unas pequeñas funciones de prueba para generar tiradas inline

In [1]:
def makelabel(label):
    if label == '':
        return ''
    else:
        return f'[{label}]'
def makecrit(crit):
    if crit == '':
        return ''
    else:
        return f'cs>{crit}'
    
def dice(dicenumber = '1', dicetype = '6', dicebonus = '0', label = '', crit = ''):
    return f'{dicenumber}d{dicetype}{makecrit(crit)}+{dicebonus}{makelabel(label)}'

def inroll(rolls = ['1d8+5[cortante]','3d6+3[fuego]',]):
    formula = rolls[0]
    if len(rolls)>1:
        for roll in rolls[1:]:
            if roll != '':
                formula += '+'+roll        
    return f'[[{formula}]]'

In [2]:
inroll()

'[[1d8+5[cortante]+3d6+3[fuego]]]'

In [3]:
bolafue = dice(3,10,-3,'fuego',18)
inroll([bolafue,bolafue,bolafue])

'[[3d10cs>18+-3[fuego]+3d10cs>18+-3[fuego]+3d10cs>18+-3[fuego]]]'

### La clave del asunto
Esta función reemplaza caracteres especiales de las macros para permitir operaciones anidadas

In [4]:
def leveldeeper(formula):
    formula = formula.replace('&','&amp;')
    formula = formula.replace('|','&#124;')
    formula = formula.replace(',','&#44;')
    formula = formula.replace('}','&#125;')
    return formula

### Querys y menús desplegables
Usando la función anterior, podemos anidar unos dentro de otros a placer

In [5]:
def query(message = 'valor', default = '0'):
    return f'?{{{leveldeeper(message)}|{leveldeeper(default)}}}'

In [6]:
def dropdown(message = 'select', options = ['option1',], labels = []):
    formula = f'?{{{leveldeeper(message)}'
    if len(labels) == len(options):
        for ii in range(len(options)):
            formula += f'|{leveldeeper(labels[ii])}, {leveldeeper(options[ii])}'
    else:
        for ii in range(len(options)):
            formula += f'|{leveldeeper(options[ii])}'
    formula += '}'
    return formula

In [7]:
query('bonus')

'?{bonus|0}'

In [8]:
drop3 = dropdown(options = ['option'+str(ii+1) for ii in range(3)])
print(drop3)

?{select|option1|option2|option3}


In [9]:
print(dropdown(options = [drop3 for ii in range(4)], labels = ['option'+str(ii+1) for ii in range(4)]))

?{select|option1, ?{select&#124;option1&#124;option2&#124;option3&#125;|option2, ?{select&#124;option1&#124;option2&#124;option3&#125;|option3, ?{select&#124;option1&#124;option2&#124;option3&#125;|option4, ?{select&#124;option1&#124;option2&#124;option3&#125;}


### Atributos
Permiten referenciar valores de una ficha de personaje en las fórmulas

#### CUIDADO
Los atributos se parsean antes que el resto de la expresión, así que no pueden usarse directamente, debe usarse un placeholder y después sustituirse

In [10]:
def atribute(pj,varname,label = ''):
    return f'[[@{{{pj}|{varname}}}{makelabel(label)}]]'

In [11]:
atribute('Paco', 'str_mod', 'Fuerza')

'[[@{Paco|str_mod}[Fuerza]]]'

## Ejemplo

In [12]:
tipoataque_labels = [
    'Normal',
    'Pericias',
    'Defensa max'
]
tipoataque_options = [
    '',
    '-5[pericias]',
    '-9[combate defensivo]'
]
tipoataque_msgs = [
    'Ataque normal',
    'Ataque con pericia, AC+5',
    'Ataque a la defensiva, AC+7'
]
bonusdrop = dropdown('Tipo de ataque', tipoataque_options, tipoataque_labels)
msgdrop = dropdown('Tipo de ataque (mensaje)', tipoataque_msgs, tipoataque_labels)

In [13]:
print(bonusdrop)

?{Tipo de ataque|Normal, |Pericias, -5[pericias]|Defensa max, -9[combate defensivo]}


# OJO
Es necesario utilizar un placeholder en el lugar del atributo, porque Roll20 parsea en primer lugar los atributos, y se desconfigurarían

In [14]:
atk_var = 'placeholder_atk'
roll_ballesta = inroll([
    dice(1,20,atk_var,'',19),
    '1[arma buena]',
    query('bonus'),
])
roll_arco = inroll([
    dice(1,20,atk_var,'',19),
    query('bonus'),
])
msg_ballesta = '\nCrítico: 19-20 x2'

roll_estoque = inroll([
    dice(1,20,atk_var,'',18),
    '1[arma mágica]',
    query('bonus'),
    bonusdrop,
])
msg_estoque = '\nCrítico: 18-20 x2\n' + msgdrop

roll_daga = inroll([
    dice(1,20,atk_var,'',19),
    '1[arma mágica]',
    query('bonus'),
    bonusdrop,
])
msg_daga = '\nCrítico: 19-20 x2\n' + msgdrop

In [15]:
print(roll_daga)

[[1d20cs>19+placeholder_atk+1[arma mágica]+?{bonus|0}+?{Tipo de ataque|Normal, |Pericias, -5[pericias]|Defensa max, -9[combate defensivo]}]]


In [16]:
arma_labels=[
    'Ballesta',
    'Estoque',
    'Daga'
]
arma_options = [
    'Ataque con Ballesta Pesada:\n' + roll_ballesta + msg_ballesta,
    'Ataque con Estoque Mágico:\n' + roll_estoque + msg_estoque,
    'Ataque con Daga Sagrada:\n' + roll_daga + msg_daga,
]
atk_drop = dropdown('Tipo de arma', arma_options, arma_labels)

In [17]:
print(atk_drop)

?{Tipo de arma|Ballesta, Ataque con Ballesta Pesada:
[[1d20cs>19+placeholder_atk+1[arma buena]+?{bonus&#124;0&#125;]]
Crítico: 19-20 x2|Estoque, Ataque con Estoque Mágico:
[[1d20cs>18+placeholder_atk+1[arma mágica]+?{bonus&#124;0&#125;+?{Tipo de ataque&#124;Normal&#44; &#124;Pericias&#44; -5[pericias]&#124;Defensa max&#44; -9[combate defensivo]&#125;]]
Crítico: 18-20 x2
?{Tipo de ataque (mensaje)&#124;Normal&#44; Ataque normal&#124;Pericias&#44; Ataque con pericia&amp;#44; AC+5&#124;Defensa max&#44; Ataque a la defensiva&amp;#44; AC+7&#125;|Daga, Ataque con Daga Sagrada:
[[1d20cs>19+placeholder_atk+1[arma mágica]+?{bonus&#124;0&#125;+?{Tipo de ataque&#124;Normal&#44; &#124;Pericias&#44; -5[pericias]&#124;Defensa max&#44; -9[combate defensivo]&#125;]]
Crítico: 19-20 x2
?{Tipo de ataque (mensaje)&#124;Normal&#44; Ataque normal&#124;Pericias&#44; Ataque con pericia&amp;#44; AC+5&#124;Defensa max&#44; Ataque a la defensiva&amp;#44; AC+7&#125;}


## Ahora ya sí
Con la macro terminada, se puede sustituir el placeholder por el atributo bueno

In [18]:
print(atk_drop.replace(atk_var, atribute('Goblin', 'strength_mod')))

?{Tipo de arma|Ballesta, Ataque con Ballesta Pesada:
[[1d20cs>19+[[@{Goblin|strength_mod}]]+1[arma buena]+?{bonus&#124;0&#125;]]
Crítico: 19-20 x2|Estoque, Ataque con Estoque Mágico:
[[1d20cs>18+[[@{Goblin|strength_mod}]]+1[arma mágica]+?{bonus&#124;0&#125;+?{Tipo de ataque&#124;Normal&#44; &#124;Pericias&#44; -5[pericias]&#124;Defensa max&#44; -9[combate defensivo]&#125;]]
Crítico: 18-20 x2
?{Tipo de ataque (mensaje)&#124;Normal&#44; Ataque normal&#124;Pericias&#44; Ataque con pericia&amp;#44; AC+5&#124;Defensa max&#44; Ataque a la defensiva&amp;#44; AC+7&#125;|Daga, Ataque con Daga Sagrada:
[[1d20cs>19+[[@{Goblin|strength_mod}]]+1[arma mágica]+?{bonus&#124;0&#125;+?{Tipo de ataque&#124;Normal&#44; &#124;Pericias&#44; -5[pericias]&#124;Defensa max&#44; -9[combate defensivo]&#125;]]
Crítico: 19-20 x2
?{Tipo de ataque (mensaje)&#124;Normal&#44; Ataque normal&#124;Pericias&#44; Ataque con pericia&amp;#44; AC+5&#124;Defensa max&#44; Ataque a la defensiva&amp;#44; AC+7&#125;}


## Default Templates

In [19]:
def default_temp(name = 'Ataque', fields = [['Ataque', '[[1d20]]'],]):
    result = f'&{{template:default}} {{{{name={leveldeeper(name)}}}}}'
    for row in fields:
        result += f' {{{{{row[0]}={row[1]}}}}}'
    return result

In [20]:
default_temp()

'&{template:default} {{name=Ataque}} {{Ataque=[[1d20]]}}'

In [21]:
def default_temp_opts(name = 'Ataque', message = 'select', labels= ['opt1', 'opt2'],
                      fields = [
                          [['Ataque', '[[1d20+1]]'],],
                          [['Ataque', '[[1d20+2]]'],],
                      ]):
    result = f'&{{template:default}} {{{{name={leveldeeper(name)}}}}}'
    options = []
    for ii in range(len(labels)):
        res_ii = ''
        opt_fields = fields[ii]
        for row in opt_fields:
            res_ii += f' {{{{{row[0]}={row[1]}}}}}'
        options.append(res_ii)
    result += dropdown(message, options, labels)
    return result

In [33]:
print(default_temp_opts())

&{template:default} {{name=Ataque}}?{select|opt1,  {{Ataque=[[1d20+1]]&#125;&#125;|opt2,  {{Ataque=[[1d20+2]]&#125;&#125;}


## CSS styling

In [52]:
def add_css(text, atributes):
    new_text = f'[{text}](#" style="'
    for pair in atributes:
        new_text += f'{pair[0]}:{pair[1]};'
    new_text += ')'
    return new_text

In [53]:
add_css('texto', [['color','red']])

'[texto](#" style="color:red;)'

### Ejemplo: tirada espías

In [22]:

tens = atribute('Nivel', 'tension')
car = atribute('selected', 'Carisma')
com = atribute('selected', 'Combate')
des = atribute('selected', 'Destreza')
ast = atribute('selected', 'Astucia')

In [23]:
tipo = dropdown('Tipo de Tirada', ['2d10', '3d10kh2', '3d10kl2'], ['normal', 'ventaja', 'desventaja'])

In [24]:
mods = ['car', 'com', 'des', 'ast']
caracts = ['carisma', 'combate', 'destreza', 'astucia']
tmp_mod=[f'{ref}_tmp' for ref in mods]
tirada = dropdown('Característica', ['[[' + tipo + '+' + ii +'-tens_temp]]' for ii in tmp_mod],caracts)

In [25]:
tirada_b = '[[' + tipo + '+temp_mod-tens_temp[nivel de tensión]]]'

In [26]:
def replace_temp(tirada):
    final_tirada = tirada
    def_mod = [car, com, des, ast]
    for ii in range(4):
        final_tirada = final_tirada.replace(tmp_mod[ii], def_mod[ii])
    final_tirada = final_tirada.replace('tens_temp', tens)
    return final_tirada

In [27]:
replace_temp(tirada)

'?{Característica|carisma, [[?{Tipo de Tirada&#124;normal&#44; 2d10&#124;ventaja&#44; 3d10kh2&#124;desventaja&#44; 3d10kl2&#125;+[[@{selected|Carisma}]]-[[@{Nivel|tension}]]]]|combate, [[?{Tipo de Tirada&#124;normal&#44; 2d10&#124;ventaja&#44; 3d10kh2&#124;desventaja&#44; 3d10kl2&#125;+[[@{selected|Combate}]]-[[@{Nivel|tension}]]]]|destreza, [[?{Tipo de Tirada&#124;normal&#44; 2d10&#124;ventaja&#44; 3d10kh2&#124;desventaja&#44; 3d10kl2&#125;+[[@{selected|Destreza}]]-[[@{Nivel|tension}]]]]|astucia, [[?{Tipo de Tirada&#124;normal&#44; 2d10&#124;ventaja&#44; 3d10kh2&#124;desventaja&#44; 3d10kl2&#125;+[[@{selected|Astucia}]]-[[@{Nivel|tension}]]]]}'

In [28]:
tirada_b

'[[?{Tipo de Tirada|normal, 2d10|ventaja, 3d10kh2|desventaja, 3d10kl2}+temp_mod-tens_temp[nivel de tensión]]]'

In [70]:
temp_titulo = add_css('---  temp_titulo  ---',[
    ['color','black'],
    ['background-color','white'],
    ['font-family', 'Courier New'],
    ['font-weight', 'bold'],
    ['text-align', 'center'],
    ['text-shadow', '0px 0px 1px black']
])
temp_titulo

'[---  temp_titulo  ---](#" style="color:black;background-color:white;font-family:Courier New;font-weight:bold;text-align:center;text-shadow:0px 0px 1px black;)'

In [73]:
def amaquina(text):
    return add_css(text,[
    ['font-family', 'Courier New'],
])

In [78]:
table_info = [
    ['1 o menor', 'pifia'],
    ['2-5', 'fallo y además'],
    ['6-10','fallo, pero'],
    ['11-15', 'éxito, pero'],
    ['16-20', 'éxito y además'],
    ['21 o más', 'éxito crítico']
]
table_lines =[[row[0], amaquina(row[1])] for row in table_info]
table = [['tirada', tirada_b]] + table_lines
template_generic = default_temp(temp_titulo, table)
template_generic

'&{template:default} {{name=[---  temp_titulo  ---](#" style="color:black;background-color:white;font-family:Courier New;font-weight:bold;text-align:center;text-shadow:0px 0px 1px black;)}} {{tirada=[[?{Tipo de Tirada|normal, 2d10|ventaja, 3d10kh2|desventaja, 3d10kl2}+temp_mod-tens_temp[nivel de tensión]]]}} {{1 o menor=[pifia](#" style="font-family:Courier New;)}} {{2-5=[fallo y además](#" style="font-family:Courier New;)}} {{6-10=[fallo, pero](#" style="font-family:Courier New;)}} {{11-15=[éxito, pero](#" style="font-family:Courier New;)}} {{16-20=[éxito y además](#" style="font-family:Courier New;)}} {{21 o más=[éxito crítico](#" style="font-family:Courier New;)}}'

In [80]:
template_chose = dropdown('Característica',
                          [template_generic.replace('temp_titulo', f'Tirada de {caracts[ii]}').replace('temp_mod', tmp_mod[ii]+f'[{caracts[ii]}]') for ii in range(4)],
                          caracts)
print(replace_temp(template_chose))

?{Característica|carisma, &amp;{template:default&#125; {{name=[---  Tirada de carisma  ---](#" style="color:black;background-color:white;font-family:Courier New;font-weight:bold;text-align:center;text-shadow:0px 0px 1px black;)&#125;&#125; {{tirada=[[?{Tipo de Tirada&#124;normal&#44; 2d10&#124;ventaja&#44; 3d10kh2&#124;desventaja&#44; 3d10kl2&#125;+[[@{selected|Carisma}]][carisma]-[[@{Nivel|tension}]][nivel de tensión]]]&#125;&#125; {{1 o menor=[pifia](#" style="font-family:Courier New;)&#125;&#125; {{2-5=[fallo y además](#" style="font-family:Courier New;)&#125;&#125; {{6-10=[fallo&#44; pero](#" style="font-family:Courier New;)&#125;&#125; {{11-15=[éxito&#44; pero](#" style="font-family:Courier New;)&#125;&#125; {{16-20=[éxito y además](#" style="font-family:Courier New;)&#125;&#125; {{21 o más=[éxito crítico](#" style="font-family:Courier New;)&#125;&#125;|combate, &amp;{template:default&#125; {{name=[---  Tirada de combate  ---](#" style="color:black;background-color:white;font-fami

### Ejemplo: Ataques de Yuri el pícaro

In [34]:
tipoataque_msgs = [
    'Ataque normal',
    'Ataque con pericia',
    'Ataque a la defensiva con pericias',
    'Ataque a la defensiva'
]
tipoataque_distancia_labels = ['normal', 'defensiva']
armor = 'armor_placeholder'
bon_fe = 'bonus_shield_placeholder'
tipoataque_ac = [
    f'+0: [[{armor}]]/[[{armor}+{bon_fe}[escudo de fe]]]/[[{armor}+{bon_fe}[escudo de fe]+7[Devoción a la Ley]]]',
    f'+5: [[{armor}+5]]/[[{armor}+{bon_fe}[escudo de fe]+5]]/[[{armor}+{bon_fe}[escudo de fe]+7[Devoción a la Ley]+5]]',
    f'+7: [[{armor}+7]]/[[{armor}+{bon_fe}[escudo de fe]+7]]/[[{armor}+{bon_fe}[escudo de fe]+7[Devoción a la Ley]+7]]',
    f'+2: [[{armor}+2]]/[[{armor}+{bon_fe}[escudo de fe]+2]]/[[{armor}+{bon_fe}[escudo de fe]+7[Devoción a la Ley]+2]]',
]

msg_estoque = 'Crítico: 18-20 x2'
roll_estoque_normal = inroll([
    dice(1,20,atk_var,'',18),
    '1[arma mágica]',
    query('bonus')
])
roll_estoque_per = inroll([
    dice(1,20,atk_var,'',18),
    '1[arma mágica]',
    query('bonus'),
    '-5[pericias]'
])
roll_estoque_def = inroll([
    dice(1,20,atk_var,'',18),
    '1[arma mágica]',
    query('bonus'),
    '-9[defensiva]'
])

msg_daga = 'Crítico: 19-20 x2'
roll_daga_normal = inroll([
    dice(1,20,atk_var,'',19),
    '1[arma mágica]',
    query('bonus'),
])
roll_daga_per = inroll([
    dice(1,20,atk_var,'',19),
    '1[arma mágica]',
    query('bonus'),
    '-5[pericias]',
])
roll_daga_def = inroll([
    dice(1,20,atk_var,'',19),
    '1[arma mágica]',
    query('bonus'),
    '-9[defensiva]'
])

msg_arco = 'Crítico: 19-20 x2'
roll_arco_normal = inroll([
    dice(1,20,atk_var,'',19),
    '1[arma buena]',
    query('bonus'),
])
roll_arco_def = inroll([
    dice(1,20,atk_var,'',19),
    '1[arma buena]',
    query('bonus'),
    '-4[defensiva]'
])

msg_ballesta = '\nCrítico: 19-20 x2'
roll_ballesta_normal = inroll([
    dice(1,20,atk_var,'',19),
    '1[arma buena]',
    query('bonus'),
])
roll_ballesta_def = inroll([
    dice(1,20,atk_var,'',19),
    '1[arma buena]',
    query('bonus'),
    '-4[defensiva]'
])

In [35]:
ataque_estoque = default_temp_opts('Ataque con Estoque Mágico', 'tipo', tipoataque_labels,
                      fields = [
                          [
                              ['Ataque', roll_estoque_normal],
                              ['', msg_estoque],
                              ['.', tipoataque_msgs[0]],
                              ['AC', tipoataque_ac[0]]
                          ],[
                              ['Ataque', roll_estoque_per],
                              ['', msg_estoque],
                              ['.', tipoataque_msgs[1]],
                              ['AC', tipoataque_ac[1]]
                          ],[
                              ['Ataque', roll_estoque_def],
                              ['', msg_estoque],
                              ['.', tipoataque_msgs[2]],
                              ['AC', tipoataque_ac[2]]
                          ]
                      ])

In [36]:
ataque_daga_Ilmater = default_temp_opts('Ataque con la Daga de Ilmater', 'tipo', tipoataque_labels,
                      fields = [
                          [
                              ['Ataque', roll_daga_normal],
                              ['', msg_daga],
                              ['.', tipoataque_msgs[0]],
                              ['AC', tipoataque_ac[0]]
                          ],[
                              ['Ataque', roll_daga_per],
                              ['', msg_daga],
                              ['.', tipoataque_msgs[1]],
                              ['AC', tipoataque_ac[1]]
                          ],[
                              ['Ataque', roll_daga_def],
                              ['', msg_daga],
                              ['.', tipoataque_msgs[2]],
                              ['AC', tipoataque_ac[2]]
                          ]
                      ])

In [37]:
ataque_daga = default_temp_opts('Ataque con la daga', 'tipo', tipoataque_labels,
                      fields = [
                          [
                              ['Ataque', roll_daga_normal],
                              ['', msg_daga],
                              ['.', tipoataque_msgs[0]],
                              ['AC', tipoataque_ac[0]]
                          ],[
                              ['Ataque', roll_daga_per],
                              ['', msg_daga],
                              ['.', tipoataque_msgs[1]],
                              ['AC', tipoataque_ac[1]]
                          ],[
                              ['Ataque', roll_daga_def],
                              ['', msg_daga],
                              ['.', tipoataque_msgs[2]],
                              ['AC', tipoataque_ac[2]]
                          ]
                      ])

In [38]:
msg_ballesta = 'Crítico: 19-20 x2'
ataque_ballesta = default_temp_opts('Ataque con ballesta', 'tipo', tipoataque_distancia_labels,
                      fields = [
                          [
                              ['Ataque', roll_ballesta_normal],
                              ['', msg_ballesta],
                              ['.', tipoataque_msgs[0]],
                              ['AC', tipoataque_ac[0]]
                          ],[
                              ['Ataque', roll_ballesta_def],
                              ['', msg_ballesta],
                              ['.', tipoataque_msgs[3]],
                              ['AC', tipoataque_ac[3]]
                          ]
                      ])

In [39]:
msg_arco = 'Crítico: 19-20 x2'
ataque_arco = default_temp_opts('Ataque con arco corto', 'tipo', tipoataque_distancia_labels,
                      fields = [
                          [
                              ['Ataque', roll_arco_normal],
                              ['', msg_arco],
                              ['.', tipoataque_msgs[0]],
                              ['AC', tipoataque_ac[0]]
                          ],[
                              ['Ataque', roll_arco_def],
                              ['', msg_arco],
                              ['.', tipoataque_msgs[3]],
                              ['AC', tipoataque_ac[3]]
                          ]
                      ])

In [40]:
arma_labels=[
    'Arco',
    'Daga'
]
arma_options = [
    ataque_arco,
    ataque_daga,
]
atk_drop = dropdown('Tipo de arma', arma_options, arma_labels)

In [41]:
print(atk_drop.replace(atk_var, atribute('Yuri', 'rangedattackbonus')).replace(armor, atribute('Yuri', 'armorclass')).replace(bon_fe, atribute('Yuri', 'bonus_ac_fe')))

?{Tipo de arma|Arco, &amp;{template:default&#125; {{name=Ataque con arco corto&#125;&#125;?{tipo&#124;normal&#44;  {{Ataque=[[1d20cs>19+[[@{Yuri|rangedattackbonus}]]+1[arma buena]+?{bonus&amp;#124;0&amp;#125;]]&amp;#125;&amp;#125; {{=Crítico: 19-20 x2&amp;#125;&amp;#125; {{.=Ataque normal&amp;#125;&amp;#125; {{AC=+0: [[[[@{Yuri|armorclass}]]]]/[[[[@{Yuri|armorclass}]]+[[@{Yuri|bonus_ac_fe}]][escudo de fe]]]/[[[[@{Yuri|armorclass}]]+[[@{Yuri|bonus_ac_fe}]][escudo de fe]+7[Devoción a la Ley]]]&amp;#125;&amp;#125;&#124;defensiva&#44;  {{Ataque=[[1d20cs>19+[[@{Yuri|rangedattackbonus}]]+1[arma buena]+?{bonus&amp;#124;0&amp;#125;+-4[defensiva]]]&amp;#125;&amp;#125; {{=Crítico: 19-20 x2&amp;#125;&amp;#125; {{.=Ataque a la defensiva&amp;#125;&amp;#125; {{AC=+2: [[[[@{Yuri|armorclass}]]+2]]/[[[[@{Yuri|armorclass}]]+[[@{Yuri|bonus_ac_fe}]][escudo de fe]+2]]/[[[[@{Yuri|armorclass}]]+[[@{Yuri|bonus_ac_fe}]][escudo de fe]+7[Devoción a la Ley]+2]]&amp;#125;&amp;#125;&#125;|Daga, &amp;{template:defau

## Minimo y máximo de n valores

In [42]:
def minval(values = ['1','3']):
    result = f'[[{{{values[0]}'
    for val in values[1:]:
        result += f',{val}'
    result += '}kl1]]'
    return result
def maxvalval(values = ['1','3']):
    result = f'[[{{{values[0]}'
    for val in values[1:]:
        result += f',{val}'
    result += '}kh1]]'
    return result

In [43]:
minval()

'[[{1,3}kl1]]'

### Ejemplo: conjuro de curación

In [44]:
cleric_lvl = 'cleric_lvl_placeholder'
cleric_bonus = 'cleric_bonus_placeholder'
launch_lvl = cleric_lvl + '+' + cleric_bonus

drop_ring = dropdown('Cargas del Anillo', ['0','2d6','3d6','4d6'],['0','1','2','3'])


heal_0 = inroll([
    '1',
    drop_ring
])
heal_1 = inroll([
    '1d8',
    minval([launch_lvl, 5],),
    drop_ring
])
heal_2 = inroll([
    '2d8',
    minval([launch_lvl, 10],),
    drop_ring
])
heal_3 = inroll([
    '3d8',
    minval([launch_lvl, 15],),
    drop_ring
])
heal_4 = inroll([
    '4d8',
    minval([launch_lvl, 20],),
    drop_ring
])
heal_1_group = inroll([
    '1d8',
    minval([launch_lvl, 25],),
    drop_ring
])
heal_2_group = inroll([
    '2d8',
    minval([launch_lvl, 30],),
    drop_ring
])
heal_3_group = inroll([
    '3d8',
    minval([launch_lvl, 35],),
    drop_ring
])
heal_4_group = inroll([
    '4d8',
    minval([launch_lvl, 40],),
    drop_ring
])
heal_wand = default_temp_opts('Curación con Varita',
                              'nivel',
                              ['1','2','3','4'],
                              [
                                  [['Varita de Curar Heridas Leves:', '[[1d8+1]]'],],
                                  [['Varita de Curar Heridas Moderadas:', '[[2d8+3]]'],],
                                  [['Varita de Curar Heridas Graves:', '[[3d8+5]]'],],
                                  [['Varita de Curar Heridas Críticas:', '[[4d8+7]]'],],
                              ]
                             )
heal_pot = default_temp_opts('Poción de Curación',
                              'nivel',
                              ['1','2','3','4'],
                              [
                                  [['Poción de Curar Heridas Leves:', '[[1d8+1]]'],],
                                  [['Poción de Curar Heridas Moderadas:', '[[2d8+3]]'],],
                                  [['Poción de Curar Heridas Graves:', '[[3d8+5]]'],],
                                  [['Poción de Curar Heridas Críticas:', '[[4d8+7]]'],],
                              ]
                             )
heal_spell = default_temp_opts('Conjuro de Curación',
                              'nivel',
                              ['0','1','2','3','4'],
                              [
                                  [['Curar Heridas Menores:', heal_0],],
                                  [['Curar Heridas Leves:', heal_1],],
                                  [['Curar Heridas Moderadas:', heal_2],],
                                  [['Curar Heridas Graves:', heal_3],],
                                  [['Curar Heridas Críticas:', heal_4],],
                              ]
                             )
heal_spell_group = default_temp_opts('Conjuro de Curación en Grupo',
                              'nivel',
                              ['1','2','3','4'],
                              [
                                  [['Curar Heridas Leves:', heal_1_group],],
                                  [['Curar Heridas Moderadas:', heal_2_group],],
                                  [['Curar Heridas Graves:', heal_3_group],],
                                  [['Curar Heridas Críticas:', heal_4_group],],
                              ]
                             )
heal_macro = dropdown('Tipo',
                      [heal_wand, heal_pot, heal_spell, heal_spell_group],
                      ['Varita', 'Poción', 'Conjuro', 'Conjuro de Grupo'])

In [45]:
print(heal_macro.replace(cleric_lvl, atribute('Yuri', 'cleric_lvl_caster')).replace(cleric_bonus, atribute('Yuri', 'bonus_lvl_heal')))

?{Tipo|Varita, &amp;{template:default&#125; {{name=Curación con Varita&#125;&#125;?{nivel&#124;1&#44;  {{Varita de Curar Heridas Leves:=[[1d8+1]]&amp;#125;&amp;#125;&#124;2&#44;  {{Varita de Curar Heridas Moderadas:=[[2d8+3]]&amp;#125;&amp;#125;&#124;3&#44;  {{Varita de Curar Heridas Graves:=[[3d8+5]]&amp;#125;&amp;#125;&#124;4&#44;  {{Varita de Curar Heridas Críticas:=[[4d8+7]]&amp;#125;&amp;#125;&#125;|Poción, &amp;{template:default&#125; {{name=Poción de Curación&#125;&#125;?{nivel&#124;1&#44;  {{Poción de Curar Heridas Leves:=[[1d8+1]]&amp;#125;&amp;#125;&#124;2&#44;  {{Poción de Curar Heridas Moderadas:=[[2d8+3]]&amp;#125;&amp;#125;&#124;3&#44;  {{Poción de Curar Heridas Graves:=[[3d8+5]]&amp;#125;&amp;#125;&#124;4&#44;  {{Poción de Curar Heridas Críticas:=[[4d8+7]]&amp;#125;&amp;#125;&#125;|Conjuro, &amp;{template:default&#125; {{name=Conjuro de Curación&#125;&#125;?{nivel&#124;0&#44;  {{Curar Heridas Menores:=[[1+?{Cargas del Anillo&amp;#124;0&amp;#44; 0&amp;#124;1&amp;#44; 2d6&

### Macro FX

In [46]:
color_opts = ['acid', 'blood', 'charm', 'death', 'fire', 'frost', 'holy', 'magic', 'slime', 'smoke', 'water']
color_drop = dropdown('select color', color_opts)

type_labels = ['beam', 'bomb', 'breath', 'bubbling', 'burn', 'burst', 'explode', 'glow', 'missile', 'nova', 'splatter']
type_drop = dropdown('select type', type_labels)
macro_fx ='/fx ' + type_drop + '-' + color_drop + ' @{target|Origin|token_id} @{target|Objective|token_id}'

In [47]:
print(macro_fx)

/fx ?{select type|beam|bomb|breath|bubbling|burn|burst|explode|glow|missile|nova|splatter}-?{select color|acid|blood|charm|death|fire|frost|holy|magic|slime|smoke|water} @{target|Origin|token_id} @{target|Objective|token_id}
